## Main
### polarization feedback control with EPC400 + controller

In [ ]:
%matplotlib widget

from nidaqmx import stream_readers, stream_writers, constants
from PyQt6 import QtWidgets, QtCore
import ipywidgets as widgets
import pyqtgraph as pg
import nidaqmx as ni
import pandas as pd
import numpy as np
import threading
import math
import time
import sys
import matplotlib.pyplot as plt
plt.rcParams['figure.autolayout'] = True; plt.rc('font', size=12); plt.rcParams['figure.max_open_warning'] = False
plt.rc('text', usetex=False); plt.rc('font', family='serif'); plt.rcParams['figure.figsize'] = (8,4)

from Instruments.OzOptics import ozopticsEPC400 as EPC400

## function
def calculateVertices(center, length):
    threshold = 5
    verticesClipped = []

    offsets = np.array([
        [-round(length/2,3), -round(length/2,3), -round(length/2,3)],
        [round(length/2,3), -round(length/2,3), -round(length/2,3)],
        [round(length/2,3), round(length/2,3), -round(length/2,3)],
        [-round(length/2,3), round(length/2,3), -round(length/2,3)],
        [-round(length/2,3), -round(length/2,3), round(length/2,3)],
        [round(length/2,3), -round(length/2,3), round(length/2,3)],
        [round(length/2,3), round(length/2,3), round(length/2,3)],
        [-round(length/2,3), round(length/2,3), round(length/2,3)]])
    vertices = center + offsets

    for vertex in vertices:
        vertexClipped = np.clip(vertex, -threshold, threshold)
        verticesClipped.append(vertexClipped)
    return verticesClipped

def signalGenerator():
    dataFrame = np.zeros((chanNum, samplesPerFrame), dtype=np.float64)

    samplesSlowPerPeriod = samplingRate/signalSlowFrequency; samplesFastPerPeriod = samplingRate/signalFastFrequency
    phaseSlowRemainder = 2*np.pi*(samplesPerFrame%samplesSlowPerPeriod)/samplesSlowPerPeriod; phaseFastRemainder = 2*np.pi*(samplesPerFrame%samplesFastPerPeriod)/samplesFastPerPeriod
    omegaSlowTime = 2*np.pi*signalSlowFrequency*timeBase; omegaFastTime = 2*np.pi*signalFastFrequency*timeBase

    frameNum = 0
    while True:
        phiSlowPhase = phaseSlowRemainder*frameNum; phiFastPhase = phaseFastRemainder*frameNum
        for channel in range(chanNum):
            signalSlowOffset = (signalSlowAmplitude[channel]+signalFastAmplitude[channel])-signalSlowAmplitude[channel]
            signalFastOffset = (signalSlowAmplitude[channel]+signalFastAmplitude[channel])-signalFastAmplitude[channel]

            # dataFrame[channel] = (signalSlowAmplitude[channel]+(signalSlowAmplitude[channel]*np.sin(omegaSlowTime+phiSlowPhase))) ## one sinusoidal, slow
            dataFrame[channel] = (signalSlowOffset+(signalSlowAmplitude[channel]*np.sin(omegaSlowTime+phiSlowPhase))) + (signalFastOffset+(signalFastAmplitude[channel]*np.sin(omegaFastTime+phiFastPhase))) ## two sinusoidals, fast and slow
            # dataFrame[channel] = (signalSlowOffset+(signalSlowAmplitude[channel]*np.sin(omegaSlowTime+phiSlowPhase)) )+ (signalFastOffset+(signalFastAmplitude[channel]*np.sign(np.sin(omegaFastTime+phiFastPhase)))) ## one sinusoid slow and one square wave fast
        yield dataFrame
        frameNum += 1

def callbackWritingTask(taskID, eventType, samplesNum, callbackData):
    writer.write_many_sample(next(callbackData), timeout=10.0)
    return 0

def callbackReadingTask(taskID, eventType, samplesNum, callbackData=None):
    global fringeVis, plotBufferFlag, plotBuffer
    reader.read_many_sample(readBuffer, samplesNum, timeout=constants.WAIT_INFINITELY)
    if plotFlag:  ## will only plot when plotFlag is on
        if plotBufferFlag: ## make sure that the current reading matches the length of the previous reading
            if len(readBuffer[0]) != len(plotBuffer): 
                readBuffer[0] = readBuffer[0][:len(plotBuffer)] ## length matching

                # offset = min(readBuffer[0]) ## normalize if negative
                # if offset < 0:
                #     readBuffer[0] = readBuffer[0]+min(readBuffer[0])
                # else:
                #     readBuffer[0] = readBuffer[0]-min(readBuffer[0])
        else:
            plotBufferFlag = True
        QtPlotData.setData(readBuffer[0]) ## update the plot
        # plotBuffer = readBuffer[0] ## update the reading buffer for length matching

    fringeVis = (max(readBuffer[0])-min(readBuffer[0]))/(max(readBuffer[0])+min(readBuffer[0])) ## calculate visibility

    ## add something to show the numbers in the plot
    # textHandler.setText(str(round(fringeVis,2)))
    return 0

## params
polControlFlag = True; polMeasFlag = True
phaseCalcFlag = False
plotFlag = False

controlDAQAddr = 'Dev1'
chanOut0 = '/ao0'; chanIn0 = '/ai0' ## SMF box
# chanOut0 = '/ao2'; chanIn0 = '/ai16' ## PM box

cubeLength = 0.5 ## max is 5
currentVoltage = np.array([0, 0, 0]) ## [V, V, V]

chanNum = 1
samplingRate = 100000; amplifierGain = 30
signalSlowFrequency = 100; signalSlowAmplitude = [20/amplifierGain] ## Vp not Vpp
signalFastFrequency = 500; signalFastAmplitude= [2*0/amplifierGain] ## Vp not Vpp

## buffer and all
mulFactor = 1; movWindow = 3; gainModifier = 1.5
framesPerBuffer = 10; refreshRate = 10 ## frames per buffer of each channel
samplesPerFrame = int(samplingRate//refreshRate)
readBuffer = np.zeros((chanNum, int(mulFactor*samplesPerFrame)), dtype=np.float64)
timeBase = np.arange(samplesPerFrame)/samplingRate
plotBufferFlag = False; plotBuffer = readBuffer ## flag for length matching

## Qt stuff
QtAppHandler = QtCore.QCoreApplication.instance() ## initialize Qt
if QtAppHandler is None:
    QtAppHandler = QtWidgets.QApplication(sys.argv)

QtAppHandler.setQuitOnLastWindowClosed(True)
QtWindowHandler = QtWidgets.QWidget() ## define top-level widget
QtWindowHandler.setWindowTitle('Oscilloscope')

QtPlotWidget = pg.PlotWidget(title='Fringes & Gradients') ## create plot widget
QtPlotData = QtPlotWidget.plot(pen='y', symbol='o', symbolBrush='y', symbolSize=5)

QtPlotHandler = QtPlotWidget.getPlotItem()
QtPlotHandler.showGrid(x=True, y=True, alpha=0.5)
QtPlotHandler.setLabel(axis='left', text='Voltage [V]'); QtPlotHandler.setLabel(axis='bottom', text='Sample [#]')

# textHandler = pg.TextItem(text='wazzap', anchor=(0, 0), color=(255, 255, 255))
# QtPlotHandler.addItem(textHandler)

layout = QtWidgets.QGridLayout() ## grid layout
layout.addWidget(QtPlotWidget) ## add the plot widget
QtWindowHandler.setLayout(layout) ## set layout to the QtWindowHandler
QtWindowHandler.show()

## init EPC if control on
if polControlFlag:
    PolCon = EPC400('ASRL6::INSTR')
    PolCon.setMode('DC')
    PolCon.setVoltages([0, 0, 0, 0])
    print('PolCon initialized...')

## daqmx tasks
taskOut = ni.Task(); taskIn = ni.Task()

taskIn.ai_channels.add_ai_voltage_chan(controlDAQAddr+chanIn0, min_val=-10, max_val=10, terminal_config=constants.TerminalConfiguration.RSE)
taskIn.timing.cfg_samp_clk_timing(rate=samplingRate, sample_mode=constants.AcquisitionType.CONTINUOUS) ## continuous sampling
taskIn.triggers.start_trigger.cfg_dig_edge_start_trig('ao/StartTrigger', trigger_edge=ni.constants.Edge.RISING) ## AI to start only once AO is triggered for simultaneous generation and acquisition:
taskIn.in_stream.input_buf_size = samplesPerFrame*framesPerBuffer*chanNum ## buffer is equal to the samplesPerChannel (samplesPerFrame*framesPerBuffer)

taskOut.ao_channels.add_ao_voltage_chan(controlDAQAddr+chanOut0, min_val=-10, max_val=10)
taskOut.timing.cfg_samp_clk_timing(rate=samplingRate, sample_mode=constants.AcquisitionType.CONTINUOUS) ## continuous sampling
taskOut.out_stream.output_buf_size = samplesPerFrame*framesPerBuffer*chanNum ## for some reason readBuffer size is not calculating correctly based on the amount of data we preload the output with (perhaps because below we fill the readBuffer using a for loop and not in one go?) so we must call out explicitly:

# taskOut.out_stream.regen_mode = ni.constants.RegenerationMode.DONT_ALLOW_REGENERATION
# taskOut.timing.implicit_underflow_behavior = ni.constants.UnderflowBehavior.AUSE_UNTIL_DATA_AVAILABLE # SIC!
## typo in the package
## DONT_ALLOW_REGENERATION prevents repeating of previous frame on output readBuffer underrun so instead of a warning the script will crash. its good to block the regeneration during development to ensure we don't get fooled by this behaviour (the warning on regeneration occurrence alone is confusing if you don't know that that's the default behaviour)

reader = stream_readers.AnalogMultiChannelReader(taskIn.in_stream)
writer = stream_writers.AnalogMultiChannelWriter(taskOut.out_stream)

## generate the signal, fill output readBuffer with data
outputFrameGenerator = signalGenerator() ## generate signal
for _ in range(framesPerBuffer): ## write em to the frames
    writer.write_many_sample(next(outputFrameGenerator), timeout=1)

taskIn.register_every_n_samples_acquired_into_buffer_event(int(mulFactor*samplesPerFrame), callbackReadingTask) ## call callback everytime AI acquired samplesPerFrame
taskOut.register_every_n_samples_transferred_from_buffer_event(samplesPerFrame, lambda *args: callbackWritingTask(*args[:-1], outputFrameGenerator)) ## lambda is there to smuggle outputFrameGenerator instance into the callbackWritingTask(scope, under the callbackData argument). the reason to pass the generator in is to avoid the necessity to define globals in the callbackWritingTask() to keep track of subsequent data frames generation.

## arms AI first (does not trigger yet), then start AO to trigger both at the same time
taskIn.start(); taskOut.start()
time.sleep(1) ## wait a bit

## threading functions
def polControlThread(stopEvent):
    global fringeVis, currentVoltage, cubeLength, dataVoltage, dataFringe, dataGradient

    voltageVertices = calculateVertices(currentVoltage, cubeLength) ## calculate vertices
    voltageVertices = np.vstack((currentVoltage, voltageVertices)) ## add origin to the vertices matrix

    j, h = 0, 1
    dataVoltage, dataFringe, dataGradient = [], [], []
    while not stopEvent.is_set():
        tempFringe = []
        for _, singleVertex in enumerate(voltageVertices):
            # timeInitDebug = time.time_ns() ## debug
            for chan, volt in enumerate(singleVertex): 
                PolCon.setVoltage(chan, volt) ## set voltages, one by one per channel
            # print('delay: ', round(1e-9*(time.time_ns()-timeInitDebug),5)) ## debug
            time.sleep(0.01)
            tempFringe.append(fringeVis)

        ## finer steps
        if j >= movWindow:
            print('cube length calc [movWindow, current]: ', round(np.mean(dataFringe[-movWindow:]),4), round(dataFringe[-1],4))
            
            ## get and check the gradient, calculated with vandermonde
            vandermondeMat = np.vander(np.arange(0,movWindow),2)
            polyCoeffs, _, _, _ = np.linalg.lstsq(vandermondeMat, dataFringe[-movWindow:], rcond=None)
            tempGradient = polyCoeffs[0]

            print('modifying cube length by: ', round(gainModifier*tempGradient,4))
            cubeLength -= gainModifier*tempGradient
        else:
            vandermondeMat = np.vander(np.arange(0,len(dataFringe)),2)
            polyCoeffs, _, _, _ = np.linalg.lstsq(vandermondeMat, dataFringe, rcond=None)
            tempGradient = polyCoeffs[0]

        ## make sure cubeLength is positive
        if cubeLength <= 0: cubeLength = 0
        
        ## append all
        nextIndex = tempFringe.index(max(tempFringe))
        currentVoltage = voltageVertices[nextIndex]
        currentFringe = tempFringe[nextIndex]

        ## debug
        print('step (next cube length, current voltage): ', round(cubeLength, 4), currentVoltage)
        print('current (fringe, gradient): ', round(currentFringe, 4), round(tempGradient, 4))
        print('\n')

        dataVoltage.append(currentVoltage); dataFringe.append(currentFringe); dataGradient.append(tempGradient)
        voltageVertices = calculateVertices(currentVoltage, cubeLength) ## recalculate the vertices based on the new origin
        voltageVertices = np.vstack((currentVoltage, voltageVertices)) ## add origin to the vertices matrix
        
        j += 1

def polMeasThread(stopEvent):
    global fringeVis, dataElapsedExport, dataFringeExport, dataGradientExport, fileName
    dataElapsedExport, dataFringeExport, dataGradientExport = [], [], []
    
    fileName = time.strftime("%y%m%d-%H%M%S")
    timeInit = time.time_ns()

    j = 0
    while not stopEvent.is_set():
        dataElapsedExport.append(round(1e-9*(time.time_ns()-timeInit),9))
        dataFringeExport.append(fringeVis)

        if j >= movWindow:
            vandermondeMat = np.vander(np.arange(0,movWindow), 2)
            polyCoeffs, _, _, _ = np.linalg.lstsq(vandermondeMat, dataFringeExport[-movWindow:], rcond=None)
            tempGradientExport = polyCoeffs[0]
            dataGradientExport.append(tempGradientExport)
        else:
            vandermondeMat = np.vander(np.arange(0,len(dataFringeExport)),2)
            polyCoeffs, _, _, _ = np.linalg.lstsq(vandermondeMat, dataFringeExport, rcond=None)
            tempGradientExport = polyCoeffs[0]
            dataGradientExport.append(tempGradientExport)
        
        j += 1
        if polMeasFlag: print('elapsed, current fringe, current gradient: ', round(dataElapsedExport[-1], 2), round(dataFringeExport[-1], 2), round(dataGradientExport[-1], 2))
        time.sleep(0.5)

def phaseCalcThread(stopEvent):
    global readBuffer, phaseAve, dataElapsedExport, dataPhaseExport, fileName
    mainBuffer, dataElapsedExport, dataPhaseExport = [], [], []
    frameNum, phiAve = 0, 0
    phi = []

    fileName = time.strftime("%y%m%d-%H%M%S")
    timeInit = time.time_ns()

    while not stopEvent.is_set():
        ## real time phase
        mainBuffer, data = [], readBuffer[0]
        cycleToRead = int(signalFastFrequency//refreshRate) ## split the frame into cycles of the waveform
        pointsPerCycle = int(samplesPerFrame//cycleToRead) ## split the frame into cycles of the waveform
        pulseOffset = int(pointsPerCycle//4) ## offset for LOW, HIGH is 3x this value

        # print('cycleToRead', cycleToRead, ' pointsPerCycle', pointsPerCycle)
        cycleNum = 0
        while cycleNum < cycleToRead: ## split the frame into cycles of the waveform
            # print('cycleNum', cycleNum)
            meanLow, meanHigh, nAverage = 0, 0, -1
            cycleSampleStart = cycleNum*pointsPerCycle

            while nAverage <= 1: ## do averaging for n window for each LOW and HIGH
                # print('cycleSampleStart', cycleSampleStart, ' pulseOffset', pulseOffset, ' nAverage', nAverage)

                ## shift the index to the beginning of the cycle + low or high offset + n window of average
                meanLow += data[cycleSampleStart + pulseOffset + nAverage] ## LOW of the waveform
                meanHigh += data[cycleSampleStart + 3*pulseOffset + nAverage] ## HIGH of the waveform
                nAverage += 1 ## next data point in the n window
            meanLow /= 3; meanHigh /= 3 ## averaging
            mainBuffer.append(meanLow); mainBuffer.append(meanHigh)

            if frameNum == 0: ## init for the first iteration
                phiInit = math.atan(mainBuffer[0]/mainBuffer[1]); alpha = phiInit ## initial alpha and Phi
                phaseAve, nPhaseJump = 0, 0
                phi.append(0)
            # else:
            #     IQtoUnpack = len(mainBuffer) >> 1 ## len(mainBuffer)/2
            #     nIQ = 0

            #     while nIQ < IQtoUnpack: ## array of InQn
            #         alphaPrevious = alpha ## carry alpha from previous
            #         inphase = mainBuffer[nIQ]; quadrature = mainBuffer[nIQ+1]
            #         alpha = math.atan(inphase/quadrature)

            #         ## correct phase jumps
            #         if alphaPrevious > 0 and (alphaPrevious-alpha) > np.pi/2: 
            #             nPhaseJump += 1; 
            #             print('jumped!')
            #         if alphaPrevious < 0 and (alpha-alphaPrevious) > np.pi/2: 
            #             nPhaseJump -= 1; 
            #             print('jumped!')

            #         # phi.append(alpha + nPhaseJump*np.pi - phiInit)
            #         phi.append(alpha + nPhaseJump*np.pi)
            #         nIQ += 2
            cycleNum += 1; frameNum += 1 ## next cycle
            print('deconstruct the frame and compile takes: ', round(1e-9*(time.time_ns()-timeInit),9))

        IQtoUnpack = len(mainBuffer) >> 1 ## len(mainBuffer)/2
        nIQ = 0

        while nIQ < IQtoUnpack: ## array of InQn
            alphaPrevious = alpha ## carry alpha from previous
            inphase = mainBuffer[nIQ]; quadrature = mainBuffer[nIQ+1]
            alpha = math.atan(inphase/quadrature)

            ## correct phase jumps
            if alphaPrevious > 0 and (alphaPrevious-alpha) > np.pi/2: 
                nPhaseJump += 1; 
                print('jumped!')
            if alphaPrevious < 0 and (alpha-alphaPrevious) > np.pi/2: 
                nPhaseJump -= 1; 
                print('jumped!')

            # phi.append(alpha + nPhaseJump*np.pi - phiInit)
            phi.append(alpha + nPhaseJump*np.pi)
            nIQ += 2

        phaseAve = np.mean(phi)
        dataElapsedExport.append(round(1e-9*(time.time_ns()-timeInit),9))
        dataPhaseExport.append(phaseAve)

        if phaseCalcFlag: print('elapsed, phaseAve: ', round(dataElapsedExport[-1], 2), round(dataPhaseExport[-1], 4))
        time.sleep(1)

## threading
threadsHandler = []
stopEvent = threading.Event()
if polControlFlag: threadsHandler.append(threading.Thread(target=polControlThread, args=(stopEvent,)))
if polMeasFlag: threadsHandler.append(threading.Thread(target=polMeasThread, args=(stopEvent,)))
if phaseCalcFlag: threadsHandler.append(threading.Thread(target=phaseCalcThread, args=(stopEvent,)))

for i in threadsHandler:
    i.start()

QtAppHandler.exec(); stopEvent.set() ## stop everything when the window is closed

taskIn.stop(); taskOut.stop()
taskIn.close(); taskOut.close()

## close the device if polControlFlag on
if polControlFlag:
    PolCon.setVoltages([0, 0, 0, 0])
    PolCon.closeDevice()

## save to file if flags on
if polMeasFlag:
    print('saving to file...')
    dataFrame = pd.DataFrame()
    dataFrame = pd.concat([dataFrame, pd.DataFrame(np.column_stack((dataElapsedExport, dataFringeExport, dataGradientExport)), columns=['Elapsed Time [s]', 'Fringe Visibility', 'Gradient'])])
    dataFrame.to_csv(fileName + '.csv', index=False, header=True) ## csv

if phaseCalcFlag:
    print('saving to file...')
    dataFrame = pd.DataFrame()
    dataFrame = pd.concat([dataFrame, pd.DataFrame(np.column_stack((dataElapsedExport, dataPhaseExport)), columns=['Elapsed Time [s]', 'Phase [rad]'])])
    dataFrame.to_csv(fileName + '.csv', index=False, header=True) ## csv

## plotting
print('plotting...')
figHandler, axHandler = plt.subplots(1, 1, layout='constrained')
if polMeasFlag:
    axHandler.plot(dataElapsedExport, dataFringeExport, '.-r', label='Data'); axHandler.set_ylabel('Fringe Visibility', color='r')
    axHandler.set_xlabel('Elapsed Time [s]')
elif polControlFlag:
    axHandler.plot(dataFringe, '.-r', label='Data'); axHandler.set_ylabel('Fringe Visibility', color='r')
    axHandler.set_xlabel('Step #')
elif phaseCalcFlag:
    axHandler.plot(dataElapsedExport, dataPhaseExport, '.-r', label='Data'); axHandler.set_ylabel('Phase [rad]', color='r')
    axHandler.set_xlabel('Elapsed Time [s]')
axHandler.tick_params(axis='y', labelcolor='r')
plt.grid(True)

if polMeasFlag:
    axHandler = axHandler.twinx()
    axHandler.plot(dataElapsedExport, dataGradientExport, '.-k', label='Data'); axHandler.set_ylabel('Gradient / Cube Length Modifier')
    axHandler.set_xlabel('Elapsed Time [s]')
elif polControlFlag:
    axHandler = axHandler.twinx()
    axHandler.plot(dataGradient, '.-k', label='Data'); axHandler.set_ylabel('Gradient / Cube Length Modifier')
    axHandler.set_xlabel('Step #')
plt.show()

### polarization feedback control with EPC400 + DAQ

In [ ]:
%matplotlib widget

from nidaqmx import stream_readers, stream_writers, constants
from PyQt6 import QtWidgets, QtCore
import ipywidgets as widgets
import pyqtgraph as pg
import nidaqmx as ni
import pandas as pd
import numpy as np
import threading
import math
import time
import sys
import matplotlib.pyplot as plt
plt.rcParams['figure.autolayout'] = True; plt.rc('font', size=12); plt.rcParams['figure.max_open_warning'] = False
plt.rc('text', usetex=False); plt.rc('font', family='serif'); plt.rcParams['figure.figsize'] = (8,4)

## function
def calculateVertices(center, length):
    threshold = 5
    verticesClipped = []

    offsets = np.array([
        [-round(length/2,3), -round(length/2,3), -round(length/2,3)],
        [round(length/2,3), -round(length/2,3), -round(length/2,3)],
        [round(length/2,3), round(length/2,3), -round(length/2,3)],
        [-round(length/2,3), round(length/2,3), -round(length/2,3)],
        [-round(length/2,3), -round(length/2,3), round(length/2,3)],
        [round(length/2,3), -round(length/2,3), round(length/2,3)],
        [round(length/2,3), round(length/2,3), round(length/2,3)],
        [-round(length/2,3), round(length/2,3), round(length/2,3)]])
    vertices = center + offsets

    for vertex in vertices:
        vertexClipped = np.clip(vertex, -threshold, threshold)
        verticesClipped.append(vertexClipped)
    return verticesClipped

def signalGenerator():
    dataFrame = np.zeros((chanNum, samplesPerFrame), dtype=np.float64)

    samplesSlowPerPeriod = samplingRate/signalSlowFrequency; samplesFastPerPeriod = samplingRate/signalFastFrequency
    phaseSlowRemainder = 2*np.pi*(samplesPerFrame%samplesSlowPerPeriod)/samplesSlowPerPeriod; phaseFastRemainder = 2*np.pi*(samplesPerFrame%samplesFastPerPeriod)/samplesFastPerPeriod
    omegaSlowTime = 2*np.pi*signalSlowFrequency*timeBase; omegaFastTime = 2*np.pi*signalFastFrequency*timeBase

    frameNum = 0
    while True:
        phiSlowPhase = phaseSlowRemainder*frameNum; phiFastPhase = phaseFastRemainder*frameNum
        for channel in range(chanNum):
            signalSlowOffset = (signalSlowAmplitude[channel]+signalFastAmplitude[channel])-signalSlowAmplitude[channel]
            signalFastOffset = (signalSlowAmplitude[channel]+signalFastAmplitude[channel])-signalFastAmplitude[channel]

            # dataFrame[channel] = (signalSlowAmplitude[channel]+(signalSlowAmplitude[channel]*np.sin(omegaSlowTime+phiSlowPhase))) ## one sinusoidal, slow
            dataFrame[channel] = (signalSlowOffset+(signalSlowAmplitude[channel]*np.sin(omegaSlowTime+phiSlowPhase))) + (signalFastOffset+(signalFastAmplitude[channel]*np.sin(omegaFastTime+phiFastPhase))) ## two sinusoidals, fast and slow
            # dataFrame[channel] = (signalSlowOffset+(signalSlowAmplitude[channel]*np.sin(omegaSlowTime+phiSlowPhase)) )+ (signalFastOffset+(signalFastAmplitude[channel]*np.sign(np.sin(omegaFastTime+phiFastPhase)))) ## one sinusoid slow and one square wave fast
        yield dataFrame
        frameNum += 1

def callbackWritingTask(taskID, eventType, samplesNum, callbackData):
    writer.write_many_sample(next(callbackData), timeout=10.0)
    return 0

def callbackReadingTask(taskID, eventType, samplesNum, callbackData=None):
    global fringeVis, plotBufferFlag, plotBuffer
    reader.read_many_sample(readBuffer, samplesNum, timeout=constants.WAIT_INFINITELY)
    if plotFlag:  ## will only plot when plotFlag is on
        if plotBufferFlag: ## make sure that the current reading matches the length of the previous reading
            if len(readBuffer[0]) != len(plotBuffer): 
                readBuffer[0] = readBuffer[0][:len(plotBuffer)] ## length matching

                # offset = min(readBuffer[0]) ## normalize
                # if offset < 0:
                #     readBuffer[0] = readBuffer[0]+min(readBuffer[0])
                # else:
                #     readBuffer[0] = readBuffer[0]-min(readBuffer[0])
        else:
            plotBufferFlag = True
        QtPlotData.setData(readBuffer[0]) ## update the plot
        # plotBuffer = readBuffer[0] ## update the reading buffer for length matching

    fringeVis = abs((max(readBuffer[0])-min(readBuffer[0]))/(max(readBuffer[0])+min(readBuffer[0]))) ## calculate visibility

    ## add something to show the numbers in the plot
    # textHandler.setText(str(round(fringeVis,2)))
    return 0

def applyVoltDAQ(task, volt):
    task.write([volt], auto_start=True)

## params
polControlFlag = True; polMeasFlag = False
phaseCalcFlag = False
plotFlag = False

controlDAQAddr = 'Dev1'
chanOut0 = '/ao0'; chanIn0 = '/ai0' ## SMF box
# chanOut0 = '/ao2'; chanIn0 = '/ai16' ## PM box
polConDAQ1Addr = 'Dev2'; polConDAQ2Addr = 'Dev3'

cubeLength = 0.7 ## max is 5
currentVoltage = np.array([0, 0, 0]) ## [V, V, V]

chanNum = 1
samplingRate = 100000; amplifierGain = 30
signalSlowFrequency = 100; signalSlowAmplitude = [20/amplifierGain] ## Vp not Vpp
signalFastFrequency = 500; signalFastAmplitude= [2*0/amplifierGain] ## Vp not Vpp

## buffer and all
mulFactor = 1; movWindow = 3; gainModifier = 1.5
framesPerBuffer = 10; refreshRate = 10 ## frames per buffer of each channel
samplesPerFrame = int(samplingRate//refreshRate)
readBuffer = np.zeros((chanNum, int(mulFactor*samplesPerFrame)), dtype=np.float64); dataBuffer = readBuffer
timeBase = np.arange(samplesPerFrame)/samplingRate
plotBufferFlag = False; plotBuffer = readBuffer ## flag for length matching

## Qt stuff
QtAppHandler = QtCore.QCoreApplication.instance() ## initialize Qt
if QtAppHandler is None:
    QtAppHandler = QtWidgets.QApplication(sys.argv)

QtAppHandler.setQuitOnLastWindowClosed(True)
QtWindowHandler = QtWidgets.QWidget() ## define top-level widget
QtWindowHandler.setWindowTitle('Oscilloscope')

QtPlotWidget = pg.PlotWidget(title='Fringes & Gradients') ## create plot widget
QtPlotData = QtPlotWidget.plot(pen='y', symbol='o', symbolBrush='y', symbolSize=5)

QtPlotHandler = QtPlotWidget.getPlotItem()
QtPlotHandler.showGrid(x=True, y=True, alpha=0.5)
QtPlotHandler.setLabel(axis='left', text='Voltage [V]'); QtPlotHandler.setLabel(axis='bottom', text='Sample [#]')

# textHandler = pg.TextItem(text='wazzap', anchor=(0, 0), color=(255, 255, 255))
# QtPlotHandler.addItem(textHandler)

layout = QtWidgets.QGridLayout() ## grid layout
layout.addWidget(QtPlotWidget) ## add the plot widget
QtWindowHandler.setLayout(layout) ## set layout to the QtWindowHandler
QtWindowHandler.show()

## init EPC if control on
if polControlFlag:
    polConDAQ1AO0Handler = ni.Task(); polConDAQ1AO1Handler = ni.Task(); polConDAQ2AO0Handler = ni.Task(); polConDAQ2AO1Handler = ni.Task()
    polConDAQ1AO0Handler.ao_channels.add_ao_voltage_chan(polConDAQ1Addr+'/ao0', min_val=-5.5, max_val=5.5); polConDAQ1AO1Handler.ao_channels.add_ao_voltage_chan(polConDAQ1Addr+'/ao1', min_val=-5.5, max_val=5.5)
    polConDAQ2AO0Handler.ao_channels.add_ao_voltage_chan(polConDAQ2Addr+'/ao0', min_val=-5.5, max_val=5.5); polConDAQ2AO1Handler.ao_channels.add_ao_voltage_chan(polConDAQ2Addr+'/ao1', min_val=-5.5, max_val=5.5)
    print('PolCon initialized...')

## daqmx tasks
taskOut = ni.Task(); taskIn = ni.Task()

taskIn.ai_channels.add_ai_voltage_chan(controlDAQAddr+chanIn0, min_val=-10, max_val=10, terminal_config=constants.TerminalConfiguration.RSE)
taskIn.timing.cfg_samp_clk_timing(rate=samplingRate, sample_mode=constants.AcquisitionType.CONTINUOUS) ## continuous sampling
taskIn.triggers.start_trigger.cfg_dig_edge_start_trig('ao/StartTrigger', trigger_edge=ni.constants.Edge.RISING) ## AI to start only once AO is triggered for simultaneous generation and acquisition:
taskIn.in_stream.input_buf_size = samplesPerFrame*framesPerBuffer*chanNum ## buffer is equal to the samplesPerChannel (samplesPerFrame*framesPerBuffer)

taskOut.ao_channels.add_ao_voltage_chan(controlDAQAddr+chanOut0, min_val=-10, max_val=10)
taskOut.timing.cfg_samp_clk_timing(rate=samplingRate, sample_mode=constants.AcquisitionType.CONTINUOUS) ## continuous sampling
taskOut.out_stream.output_buf_size = samplesPerFrame*framesPerBuffer*chanNum ## for some reason readBuffer size is not calculating correctly based on the amount of data we preload the output with (perhaps because below we fill the readBuffer using a for loop and not in one go?) so we must call out explicitly:

# taskOut.out_stream.regen_mode = ni.constants.RegenerationMode.DONT_ALLOW_REGENERATION
# taskOut.timing.implicit_underflow_behavior = ni.constants.UnderflowBehavior.AUSE_UNTIL_DATA_AVAILABLE # SIC!
## typo in the package
## DONT_ALLOW_REGENERATION prevents repeating of previous frame on output readBuffer underrun so instead of a warning the script will crash. its good to block the regeneration during development to ensure we don't get fooled by this behaviour (the warning on regeneration occurrence alone is confusing if you don't know that that's the default behaviour)

reader = stream_readers.AnalogMultiChannelReader(taskIn.in_stream)
writer = stream_writers.AnalogMultiChannelWriter(taskOut.out_stream)

## generate the signal, fill output readBuffer with data
outputFrameGenerator = signalGenerator() ## generate signal
for _ in range(framesPerBuffer): ## write em to the frames
    writer.write_many_sample(next(outputFrameGenerator), timeout=1)

taskIn.register_every_n_samples_acquired_into_buffer_event(int(mulFactor*samplesPerFrame), callbackReadingTask) ## call callback everytime AI acquired samplesPerFrame
taskOut.register_every_n_samples_transferred_from_buffer_event(samplesPerFrame, lambda *args: callbackWritingTask(*args[:-1], outputFrameGenerator)) ## lambda is there to smuggle outputFrameGenerator instance into the callbackWritingTask(scope, under the callbackData argument). the reason to pass the generator in is to avoid the necessity to define globals in the callbackWritingTask() to keep track of subsequent data frames generation.

## arms AI first (does not trigger yet), then start AO to trigger both at the same time
taskIn.start(); taskOut.start()
time.sleep(1) ## wait a bit





## threading functions
def polControlThread(stopEvent):
    global fringeVis, currentVoltage, cubeLength, dataVoltage, dataFringe, dataGradient
    
    applyVoltDAQ(polConDAQ2AO1Handler, 0); time.sleep(0.0125) ## apply 0V to the last channel
    voltageVertices = calculateVertices(currentVoltage, cubeLength) ## calculate vertices
    voltageVertices = np.vstack((currentVoltage, voltageVertices)) ## add origin to the vertices matrix

    j, h = 0, 1
    dataVoltage, dataFringe, dataGradient = [], [], []
    while not stopEvent.is_set():
        tempFringe = []
        for _, singleVertex in enumerate(voltageVertices):
            # timeInitDebug = time.time_ns() ## debug
            # for chan, volt in enumerate(singleVertex):
                # PolCon.setVoltage(chan, volt) 
                
            ## set voltages, one by one per channel
            
            applyVoltDAQ(polConDAQ1AO0Handler, singleVertex[0]); time.sleep(0.0125)
            applyVoltDAQ(polConDAQ1AO1Handler, singleVertex[1]); time.sleep(0.0125)
            applyVoltDAQ(polConDAQ2AO0Handler, singleVertex[2]); time.sleep(0.0125)

            # print('delay: ', round(1e-9*(time.time_ns()-timeInitDebug),5)) ## debug
            time.sleep(0.1)
            tempFringe.append(fringeVis)

        ## finer steps
        if j >= movWindow:
            print('cube length calc [movWindow, current]: ', round(np.mean(dataFringe[-movWindow:]),4), round(dataFringe[-1],4))
            
            ## get and check the gradient, calculated with vandermonde
            vandermondeMat = np.vander(np.arange(0,movWindow),2)
            polyCoeffs, _, _, _ = np.linalg.lstsq(vandermondeMat, dataFringe[-movWindow:], rcond=None)
            tempGradient = polyCoeffs[0]

            print('modifying cube length by: ', round(gainModifier*tempGradient,4))
            cubeLength -= gainModifier*tempGradient
        else:
            vandermondeMat = np.vander(np.arange(0,len(dataFringe)),2)
            polyCoeffs, _, _, _ = np.linalg.lstsq(vandermondeMat, dataFringe, rcond=None)
            tempGradient = polyCoeffs[0]

        ## make sure cubeLength is positive
        if cubeLength <= 0: cubeLength = 0
        
        ## append all
        nextIndex = tempFringe.index(max(tempFringe))
        currentVoltage = voltageVertices[nextIndex]
        currentFringe = tempFringe[nextIndex]

        ## debug
        print('step (next cube length, current voltage): ', round(cubeLength, 4), currentVoltage)
        print('current (fringe, gradient): ', round(currentFringe, 4), round(tempGradient, 4))
        print('\n')

        dataVoltage.append(currentVoltage); dataFringe.append(currentFringe); dataGradient.append(tempGradient)
        voltageVertices = calculateVertices(currentVoltage, cubeLength) ## recalculate the vertices based on the new origin
        voltageVertices = np.vstack((currentVoltage, voltageVertices)) ## add origin to the vertices matrix
        
        j += 1

def polMeasThread(stopEvent):
    global fringeVis, dataElapsedExport, dataFringeExport, dataGradientExport, fileName
    dataElapsedExport, dataFringeExport, dataGradientExport = [], [], []
    
    fileName = time.strftime("%y%m%d-%H%M%S")
    timeInit = time.time_ns()

    j = 0
    while not stopEvent.is_set():
        dataElapsedExport.append(round(1e-9*(time.time_ns()-timeInit),9))
        dataFringeExport.append(fringeVis)

        if j >= movWindow:
            vandermondeMat = np.vander(np.arange(0,movWindow), 2)
            polyCoeffs, _, _, _ = np.linalg.lstsq(vandermondeMat, dataFringeExport[-movWindow:], rcond=None)
            tempGradientExport = polyCoeffs[0]
            dataGradientExport.append(tempGradientExport)
        else:
            vandermondeMat = np.vander(np.arange(0,len(dataFringeExport)),2)
            polyCoeffs, _, _, _ = np.linalg.lstsq(vandermondeMat, dataFringeExport, rcond=None)
            tempGradientExport = polyCoeffs[0]
            dataGradientExport.append(tempGradientExport)
        
        j += 1
        if polMeasFlag: print('elapsed, current fringe, current gradient: ', round(dataElapsedExport[-1], 2), round(dataFringeExport[-1], 2), round(dataGradientExport[-1], 2))
        time.sleep(0.5)

def phaseCalcThread(stopEvent):
    global readBuffer, phaseAve, dataElapsedExport, dataPhaseExport, fileName
    mainBuffer, dataElapsedExport, dataPhaseExport = [], [], []
    frameNum, phiAve = 0, 0
    phi = []

    fileName = time.strftime("%y%m%d-%H%M%S")
    timeInit = time.time_ns()

    while not stopEvent.is_set():
        ## real time phase
        mainBuffer, data = [], readBuffer[0]
        cycleToRead = int(signalFastFrequency//refreshRate) ## split the frame into cycles of the waveform
        pointsPerCycle = int(samplesPerFrame//cycleToRead) ## split the frame into cycles of the waveform
        pulseOffset = int(pointsPerCycle//4) ## offset for LOW, HIGH is 3x this value

        # print('cycleToRead', cycleToRead, ' pointsPerCycle', pointsPerCycle)
        cycleNum = 0
        while cycleNum < cycleToRead: ## split the frame into cycles of the waveform
            # print('cycleNum', cycleNum)
            meanLow, meanHigh, nAverage = 0, 0, -1
            cycleSampleStart = cycleNum*pointsPerCycle

            while nAverage <= 1: ## do averaging for n window for each LOW and HIGH
                # print('cycleSampleStart', cycleSampleStart, ' pulseOffset', pulseOffset, ' nAverage', nAverage)

                ## shift the index to the beginning of the cycle + low or high offset + n window of average
                meanLow += data[cycleSampleStart + pulseOffset + nAverage] ## LOW of the waveform
                meanHigh += data[cycleSampleStart + 3*pulseOffset + nAverage] ## HIGH of the waveform
                nAverage += 1 ## next data point in the n window
            meanLow /= 3; meanHigh /= 3 ## averaging
            mainBuffer.append(meanLow); mainBuffer.append(meanHigh)

            if frameNum == 0: ## init for the first iteration
                phiInit = math.atan(mainBuffer[0]/mainBuffer[1]); alpha = phiInit ## initial alpha and Phi
                phaseAve, nPhaseJump = 0, 0
                phi.append(0)
            # else:
            #     IQtoUnpack = len(mainBuffer) >> 1 ## len(mainBuffer)/2
            #     nIQ = 0

            #     while nIQ < IQtoUnpack: ## array of InQn
            #         alphaPrevious = alpha ## carry alpha from previous
            #         inphase = mainBuffer[nIQ]; quadrature = mainBuffer[nIQ+1]
            #         alpha = math.atan(inphase/quadrature)

            #         ## correct phase jumps
            #         if alphaPrevious > 0 and (alphaPrevious-alpha) > np.pi/2: 
            #             nPhaseJump += 1; 
            #             print('jumped!')
            #         if alphaPrevious < 0 and (alpha-alphaPrevious) > np.pi/2: 
            #             nPhaseJump -= 1; 
            #             print('jumped!')

            #         # phi.append(alpha + nPhaseJump*np.pi - phiInit)
            #         phi.append(alpha + nPhaseJump*np.pi)
            #         nIQ += 2
            cycleNum += 1; frameNum += 1 ## next cycle
            print('deconstruct the frame and compile takes: ', round(1e-9*(time.time_ns()-timeInit),9))

        IQtoUnpack = len(mainBuffer) >> 1 ## len(mainBuffer)/2
        nIQ = 0

        while nIQ < IQtoUnpack: ## array of InQn
            alphaPrevious = alpha ## carry alpha from previous
            inphase = mainBuffer[nIQ]; quadrature = mainBuffer[nIQ+1]
            alpha = math.atan(inphase/quadrature)

            ## correct phase jumps
            if alphaPrevious > 0 and (alphaPrevious-alpha) > np.pi/2: 
                nPhaseJump += 1; 
                print('jumped!')
            if alphaPrevious < 0 and (alpha-alphaPrevious) > np.pi/2: 
                nPhaseJump -= 1; 
                print('jumped!')

            # phi.append(alpha + nPhaseJump*np.pi - phiInit)
            phi.append(alpha + nPhaseJump*np.pi)
            nIQ += 2

        phaseAve = np.mean(phi)
        dataElapsedExport.append(round(1e-9*(time.time_ns()-timeInit),9))
        dataPhaseExport.append(phaseAve)

        if phaseCalcFlag: print('elapsed, phaseAve: ', round(dataElapsedExport[-1], 2), round(dataPhaseExport[-1], 4))
        time.sleep(1)

## threading
threadsHandler = []
stopEvent = threading.Event()
if polControlFlag: threadsHandler.append(threading.Thread(target=polControlThread, args=(stopEvent,)))
if polMeasFlag: threadsHandler.append(threading.Thread(target=polMeasThread, args=(stopEvent,)))
if phaseCalcFlag: threadsHandler.append(threading.Thread(target=phaseCalcThread, args=(stopEvent,)))

for i in threadsHandler:
    i.start()

QtAppHandler.exec(); stopEvent.set() ## stop everything when the window is closed

taskIn.stop(); taskOut.stop()
taskIn.close(); taskOut.close()

## close the device if polControlFlag on
if polControlFlag:
    polConDAQ1AO0Handler.stop(); polConDAQ1AO0Handler.close()
    polConDAQ1AO1Handler.stop(); polConDAQ1AO1Handler.close()
    polConDAQ2AO0Handler.stop(); polConDAQ2AO0Handler.close()
    polConDAQ2AO1Handler.stop(); polConDAQ2AO1Handler.close()














## save to file if flags on
if polMeasFlag:
    print('saving to file...')
    dataFrame = pd.DataFrame()
    dataFrame = pd.concat([dataFrame, pd.DataFrame(np.column_stack((dataElapsedExport, dataFringeExport, dataGradientExport)), columns=['Elapsed Time [s]', 'Fringe Visibility', 'Gradient'])])
    dataFrame.to_csv(fileName + '.csv', index=False, header=True) ## csv

if phaseCalcFlag:
    print('saving to file...')
    dataFrame = pd.DataFrame()
    dataFrame = pd.concat([dataFrame, pd.DataFrame(np.column_stack((dataElapsedExport, dataPhaseExport)), columns=['Elapsed Time [s]', 'Phase [rad]'])])
    dataFrame.to_csv(fileName + '.csv', index=False, header=True) ## csv

## plotting
print('plotting...')
figHandler, axHandler = plt.subplots(1, 1, layout='constrained')
if polMeasFlag:
    axHandler.plot(dataElapsedExport, dataFringeExport, '.-r', label='Data'); axHandler.set_ylabel('Fringe Visibility', color='r')
    axHandler.set_xlabel('Elapsed Time [s]')
elif polControlFlag:
    axHandler.plot(dataFringe, '.-r', label='Data'); axHandler.set_ylabel('Fringe Visibility', color='r')
    axHandler.set_xlabel('Step #')
elif phaseCalcFlag:
    axHandler.plot(dataElapsedExport, dataPhaseExport, '.-r', label='Data'); axHandler.set_ylabel('Phase [rad]', color='r')
    axHandler.set_xlabel('Elapsed Time [s]')
axHandler.tick_params(axis='y', labelcolor='r')
plt.grid(True)

if polMeasFlag:
    axHandler = axHandler.twinx()
    axHandler.plot(dataElapsedExport, dataGradientExport, '.-k', label='Data'); axHandler.set_ylabel('Gradient / Cube Length Modifier')
    axHandler.set_xlabel('Elapsed Time [s]')
elif polControlFlag:
    axHandler = axHandler.twinx()
    axHandler.plot(dataGradient, '.-k', label='Data'); axHandler.set_ylabel('Gradient / Cube Length Modifier')
    axHandler.set_xlabel('Step #')
plt.show()

### manual control with controller

In [ ]:
%matplotlib inline

import os
import csv
import time
import psutil
import pyvisa
import random
import asyncio
import threading
import numpy as np
import pandas as pd
import nidaqmx as ni
from ctypes import *
from nidaqmx import constants
import ipywidgets as widgets
from datetime import datetime
import matplotlib.pyplot as plt
plt.rcParams['figure.autolayout'] = True; plt.rc('font', size=16)
plt.rc('text', usetex=False); plt.rc('font', family='serif'); plt.rcParams['figure.figsize'] = (10, 5)

from Instruments.OzOptics import ozopticsEPC400 as EPC400
def applyVoltFun(chan, volt):
    polControllerHandler.setVoltage(chan, volt)

## init params
resMan = pyvisa.ResourceManager()
outputMain = widgets.Output()

if len(resMan.list_resources()) > 1: instAddress = list(resMan.list_resources())[0] ## ignore if nothing found, can "simulate" from personal laptop
instHandler, polControllerHandler = 'None', 'None'
connStatus = 'Idle'

## classes

## init widgets
scanButton = widgets.Button(description='Scan for Instruments'); checkConnectButton = widgets.Button(description='Check Connection')
connectInstMeasButton = widgets.Button(description='Connect'); stopMeasButton = widgets.Button(description='Stop & Disconnect')
instIDBox = widgets.Text(); dropdownBox = widgets.Dropdown(options=list(resMan.list_resources()), description='Address')

## scan and connect
scanConnectButtonHandler = widgets.HBox([scanButton, checkConnectButton])
instFindHandler = widgets.VBox([scanConnectButtonHandler, dropdownBox])
instStartHandler = widgets.VBox([instIDBox, widgets.HBox([connectInstMeasButton, stopMeasButton])])

## first accordion 
polConConnBox = widgets.Text(description='Polarization Controller Status', value='Not Connected', style={'description_width':'initial'})
polConConnButton = widgets.Button(description='Connect', style={'description_width':'initial'}); polConAddrBox = widgets.Text(description='Polarization Controller Address', value='ASRL6::INSTR', style={'description_width':'initial'})
v1Slider = widgets.FloatSlider(description='V1 [V]', value=0, min=-5, max=5, step=0.01, continuous_update=True, readout=True, disabled=True); v2Slider = widgets.FloatSlider(description='V2 [V]', value=0, min=-5, max=5, step=0.01, continuous_update=True, readout=True, disabled=True)
v3Slider = widgets.FloatSlider(description='V3 [V]', value=0, min=-5, max=5, step=0.01, continuous_update=True, readout=True, disabled=True); v4Slider = widgets.FloatSlider(description='V4 [V]', value=0, min=-5, max=5, step=0.01, continuous_update=True, readout=True, disabled=True)
polConLogBox = widgets.Textarea(layout=widgets.Layout(height='100px', width='100%'))

polConHandler = widgets.VBox([widgets.HBox([widgets.VBox([widgets.HBox([v1Slider]), widgets.HBox([v2Slider]), 
                                        widgets.HBox([v3Slider]), widgets.HBox([v4Slider])]), 
                              widgets.HBox([widgets.VBox([polConConnBox, polConAddrBox]), 
                                            polConConnButton])
                                          ]),
                            polConLogBox
                            ])

## all accordion
AccordHandler = widgets.Accordion(children=[polConHandler])
AccordHandler.set_title(0, 'Polarization Controller: EPC400 for Feedback')

## display widgets
display(
    widgets.VBox([widgets.Label('AdvancedInterconnect: Optimized for Fiber Control'),
        widgets.HBox([instFindHandler, instStartHandler]),
        AccordHandler]), 
    outputMain)

## functions & classes
def logInBox(textbox, message):
    textbox.value = str(datetime.now()) + '   ' + message + '\n' + textbox.value

## handlers
def scanButtonHandler(value):
    with outputMain:
        dropdownBox.options = list(resMan.list_resources())
        print(str(datetime.now()) + '   Scanning for instruments...')
scanButton.on_click(scanButtonHandler)

def dropdownBoxHandler(value):
    global instAddress
    if value['type'] == 'change' and value['name'] == 'value':
        with outputMain:
            instAddress = value['new']
            print(str(datetime.now()) + '   Selected device address: ' + instAddress)
dropdownBox.observe(dropdownBoxHandler)

def checkConnectButtonHandler(value):
    global instHandler, instAddress
    with outputMain:
        connectInstMeasButton.disabled = True
        print(str(datetime.now()) + '   Connecting to: ' + instAddress)
        instHandler = resMan.open_resource(instAddress)
        instIDBox.value = instHandler.query('*IDN?').strip()
        print(str(datetime.now()) + '   Connected to: ' + instIDBox.value)
        connectInstMeasButton.disabled = False
checkConnectButton.on_click(checkConnectButtonHandler)

def connectInstButtonHandler(value):
    global instHandler, polControllerHandler
    with outputMain:
        if instIDBox.value != '':
                instHandler = resMan.open_resource(instAddress)
                print(str(datetime.now()) + '   Connection established to: ' + instIDBox.value); instIDBox.value = ''
        else:
            instIDBox.value = 'Check connection first!'
connectInstMeasButton.on_click(connectInstButtonHandler)

def stopMeasButtonHandler(value):
    global instHandler, polControllerHandler
    with outputMain:
        print(str(datetime.now()) + '   Ending measurements...')
        instIDBox.value = ''
        instHandler = 'None'; polControllerHandler = 'None'

        polConConnBox.value = 'Not Connected'
        print(str(datetime.now()) + '   Device(s) disconnected.')
        stopEvent.set()
        print(str(datetime.now()) + '   Everything is stopped. Rerun the file to start again.')
stopMeasButton.on_click(stopMeasButtonHandler)

def polConConnButtonHandler(value):
    global polConAddr, polControllerHandler
    with outputMain:
        if polConConnBox.value == 'Not Connected':
            connStatus = 'Connecting'
            polConAddr = polConAddrBox.value
            polControllerHandler = EPC400(polConAddr)
            polControllerHandler.setMode('DC')
            polControllerHandler.setVoltages([0, 0, 0, 0])
            v1Slider.value = 0; v2Slider.value = 0
            v3Slider.value = 0; v4Slider.value = 0

            v1Slider.disabled = False; v2Slider.disabled = False
            v3Slider.disabled = False; v4Slider.disabled = False

            instIDBox.value = 'EPC400 connected!'
            logInBox(polConLogBox, 'Connection established to EPC400 and voltages set to [0,0,0,0]...')
            polConConnBox.value = 'Connected'; polConConnButton.description = 'Disconnect'
            connStatus = 'Idle'
        else:
            connStatus = 'Disconnecting'
            polControllerHandler.setVoltages([0, 0, 0, 0])
            v1Slider.value = 0; v2Slider.value = 0
            v3Slider.value = 0; v4Slider.value = 0
            polControllerHandler.closeDevice()

            v1Slider.disabled = True; v2Slider.disabled = True
            v3Slider.disabled = True; v4Slider.disabled = True
            
            instIDBox.value = 'EPC400 disconnected!'
            logInBox(polConLogBox, 'EPC400 disconnected successfully...')
            polConConnBox.value = 'Not Connected'; polConConnButton.description = 'Connect'
            connStatus = 'Idle'
polConConnButton.on_click(polConConnButtonHandler)

## polarization controller slider handler
def v1Fun(valueChanged): applyVoltFun(0, valueChanged.new)
v1Slider.observe(v1Fun, 'value')
def v2Fun(valueChanged): applyVoltFun(1, valueChanged.new)
v2Slider.observe(v2Fun, 'value')
def v3Fun(valueChanged): applyVoltFun(2, valueChanged.new)
v3Slider.observe(v3Fun, 'value')
def v4Fun(valueChanged): applyVoltFun(3, valueChanged.new)
v4Slider.observe(v4Fun, 'value')

## readout functions
def updateReadout(stopEvent):
    while True:
        if connStatus == 'Idle':

            time.sleep(0.1)

## threadings
threadsHandler = []
stopEvent = threading.Event()
threadsHandler.append(threading.Thread(target=updateReadout, args=(stopEvent,)))

for i in threadsHandler:
    i.start()